In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from pyphare.pharesee.run import Run
from pyphare.pharesee.hierarchy import hierarchy_utils

mpl.use("TkAgg")

In [15]:
run_path = os.path.join(os.curdir, "nCheck")
time = 0.0

In [16]:
r = Run(run_path)

In [17]:
h = r.GetMassDensity(time)
# H = hierarchy_utils.rename(r.GetMassDensity(time), ["rho"])

In [20]:
# h.plot("rho")

In [34]:
def sigma(funcs, x):
    return sum(f(x) for f in funcs)

phi = [lambda x: x**2, lambda x: x]
x = np.array([1, 2, 3])
y = sigma(phi, x)

In [36]:
def expected_mass_density_func(x, **kwargs):
    masses = kwargs["masses"]
    densities = kwargs["densities"]

    assert len(masses) == len(densities)

    funcs = []
    for mass, density in zip(masses, densities):
        funcs.append(lambda x: mass*density(x))

    return sum(f(x) for f in funcs)

In [26]:
def hierarchy_fromfunc(func, hier, **kwargs):
    from pyphare.pharesee.hierarchy.hierarchy_utils import compute_hier_from
    # return hier

    return compute_hier_from(func, hier, **kwargs)

In [22]:
H = hierarchy_fromfunc(lambda x:x, h)

In [25]:
# hierarchy_utils.hierarchy_compare(h, h)

In [ ]:
def _compute_sqrt(patch_datas, **kwargs):
    ref_name = next(iter(patch_datas.keys()))

    dset = np.sqrt(patch_datas["value"][:])

    return (
        {"name": "value", "data": dset, "centering": patch_datas[ref_name].centerings},
    )


In [27]:
def zob(x):
    return sum(lambda x:x, lambda x : x^2)

In [28]:
zob(2)

TypeError: 'function' object is not iterable

In [31]:
def sigma(funcs, x):
    return sum(f(x) for f in funcs)

phi = [lambda x: x**2, lambda x: x]
x = np.array([1, 2, 3])
y = sigma(phi, x)

In [32]:
y

array([ 2,  6, 12])

In [4]:
Ni = hierarchy_utils.rename(r.GetMassDensity(time), ["rho"])

NameError: name 'hierarchy_utils' is not defined

In [65]:
fig, ax = plt.subplots(figsize=(8, 6))
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [66]:
Ni.plot(ax=ax, ls="-", lw=2.0, color=cycle[0])

In [67]:
plt.show()

In [68]:
dir({})

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [16]:
mass_computed = r.GetMassDensity(time)

In [8]:
Nm.level(0).patches[0].patch_datas

{'value': FieldData: (box=(Box([0],[49]), [50]), key=value)}

In [9]:
from pyphare.pharesee.hierarchy import hierarchy_utils

In [10]:
newN = hierarchy_utils.rename(Nm, ["rho"])

In [11]:
newN.level(0).patches[0].patch_datas

{'rho': FieldData: (box=(Box([0],[49]), [50]), key=rho)}

In [5]:
hier_computed = r.GetMassDensity(time)

In [6]:
L = sim.simulation_domain()

NameError: name 'sim' is not defined

In [50]:
def densityMain(x):
    return 1.

def densityBeam(x):
    u = x/L-0.5
    return np.exp(-u**2)

m_Main = 2
m_Beam = 3

In [51]:
def _compute_ions_mass_density_profile(patch_datas, **kwargs):
    ref_name = next(iter(patch_datas.keys()))

    x_ = patch_datas[ref_name].x
    print(type(x_), x_)
    dset = m_Main*densityMain(x_)+m_Beam*densityBeam(x_)
    
    # dset = patch_datas["value"][:]
    return ({"name": "value", "data": dset, "centering": patch_datas[ref_name].centerings},)

In [52]:
from pyphare.pharesee.hierarchy.hierarchy_utils import compute_hier_from
hier_expected = compute_hier_from(_compute_ions_mass_density_profile, hier_computed)

<class 'numpy.ndarray'> [-0.4 -0.2  0.   0.2  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  2.   2.2
  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8  5.
  5.2  5.4  5.6  5.8  6.   6.2  6.4  6.6  6.8  7.   7.2  7.4  7.6  7.8
  8.   8.2  8.4  8.6  8.8  9.   9.2  9.4  9.6  9.8 10.  10.2 10.4]
<class 'numpy.ndarray'> [ 9.6  9.8 10.  10.2 10.4 10.6 10.8 11.  11.2 11.4 11.6 11.8 12.  12.2
 12.4 12.6 12.8 13.  13.2 13.4 13.6 13.8 14.  14.2 14.4 14.6 14.8 15.
 15.2 15.4 15.6 15.8 16.  16.2 16.4 16.6 16.8 17.  17.2 17.4 17.6 17.8
 18.  18.2 18.4 18.6 18.8 19.  19.2 19.4 19.6 19.8 20.  20.2 20.4]


In [53]:
from pyphare.pharein import global_vars

In [54]:
global_vars.sim